<a href="https://colab.research.google.com/github/joomj2000/Chatbot/blob/main/2_prompt_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai
!pip install langchain
!pip install langchain-community langchain-core


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.8 MB/s eta 0:00:00


In [3]:
# os의 환경변수에 키값 저장
import os

os.environ["OPENAI_API_KEY"] = '64650e80fc9f42d28849c6a5f601d832'
os.environ["OPENAI_API_TYPE"] = 'azure'
os.environ["OPENAI_API_VERSION"] = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://sktflyai.openai.azure.com/'

In [8]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate  # PromptTemplate는 완성형,ChatPromptTemplate는 대화형 언어모델에서 사용

# 완성형
string_prompt= PromptTemplate.from_template("Tell me a joke about {topic}") #템플릿 저장
prompt=string_prompt.format(topic="soccer") # topic 만 바꾸기

print(prompt)

Tell me a joke about soccer


In [14]:
# 대화형
string_prompt= ChatPromptTemplate.from_template("Tell me a joke about {topic}") #템플릿 저장
prompt=string_prompt.format_messages(topic="soccer") # topic 만 바꾸기

print(prompt) # HumanNessage라고 명시되어있음
print(prompt[0].content) #메시지만 출력하기

[HumanMessage(content='Tell me a joke about soccer')]
Tell me a joke about soccer


완성형 모델 템플릿 만들기 - 레시피 생성

In [16]:
string_template='''
너는 요리사다. 제시하는 재료들로 만들 수 있는 요리를 추천하고 레시피를 제시하라

내가 가진 재료는 아래와 같다.

- 재 료 -
{ingredents}

'''

In [17]:
prompt=PromptTemplate(
    input_variables=['ingredents'],
    template=string_template ,
)

prompt

PromptTemplate(input_variables=['ingredents'], template='\n너는 요리사다. 제시하는 재료들로 만들 수 있는 요리를 추천하고 레시피를 제시하라\n\n내가 가진 재료는 아래와 같다. \n\n- 재 료 -\n{ingredents}\n\n')

In [18]:
prompt.format(ingredents="소고기, 양파, 당근")

'\n너는 요리사다. 제시하는 재료들로 만들 수 있는 요리를 추천하고 레시피를 제시하라\n\n내가 가진 재료는 아래와 같다. \n\n- 재 료 -\n소고기, 양파, 당근\n\n'

In [24]:
from langchain.llms import AzureOpenAI

llm=AzureOpenAI(
    deployment_name='dev-gpt-35-turbo-instruct' #완성형 모델
)


response=llm(prompt.format(ingredents="소고기, 양파, 당근"))
print(response)

내가 추천하는 요리는 소고기 덮밥이다.

레시피:

1. 소고기를 잘게 썰어서 양념에 넣는다. (양념은 간장, 설탕, 다진 파, 다진 마늘, 후추를 섞어서 만든다.)
2. 양파와 당근을 채 썰어 준비한다.
3. 팬에 기름을 두르고 양파와 당근을 볶는다.
4. 볶은 양파와 당근을 덮고, 소고기를 넣고 익힌다.
5. 소고기가 익으면 밥 위에 얹어준다.
6. 마지막으로 넣은 양파와 당근을 소고기 위로 올려주고 잠시 볶아준다.
7. 고기가 완전히 익으면 불을


대화형 모델 사용하기

In [25]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [26]:
from langchain.chat_models import AzureChatOpenAI

chatgpt=AzureChatOpenAI(
    deployment_name='dev-gpt-35-turbo', # 대화형 모델
    model_name='dev-gpt-35-turbo'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [27]:
# 대화형 프롬프트 템플릿 지정

system_prompt_template=SystemMessagePromptTemplate.from_template(string_template) # 페르소나 잡기
human_prompt_template=HumanMessagePromptTemplate.from_template("{ingredents}")

chat_prompt=ChatPromptTemplate.from_messages([system_prompt_template,human_prompt_template])

In [28]:
answer=chatgpt(chat_prompt.format_prompt(ingredents="소고기, 양파, 당근").to_messages())
print(answer.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


제안하는 요리는 "소고기볶음"입니다.

레시피:

1. 소고기를 얇게 썰어 준비합니다.
2. 양파와 당근을 각각 얇게 채썰어 준비합니다.
3. 팬에 식용유를 두르고 중약불에서 양파와 당근을 볶아줍니다.
4. 양파와 당근이 약간 익으면 소고기를 넣고 함께 볶아줍니다.
5. 소고기가 완전히 익고 양파와 당근이 부드러워지면 간장, 설탕, 소금, 후추를 넣고 볶아줍니다.
6. 재료들이 잘 섞이고 간이 배여지면 완성입니다. 밥과 함께 즐기세요!

소고기볶음은 간단하면서도 맛있는 요리로, 소고기의 짭짤한 맛과 양파와 당근의 고소한 맛이 어우러져 맛있게 먹을 수 있습니다. 재료들을 적절히 볶아내면서 간장, 설탕, 소금, 후추로 간을 조절해주세요. 부드러운 밥과 함께 먹으면 더욱 맛있게 즐길 수 있습니다. 즐거운 요리 시간 되세요!


few-short learning : 삼행시 짓기


In [29]:
from langchain.prompts.few_shot import FewShotPromptTemplate


examples=[
    {
        'question':'아이유로 삼행시를 지어줘',
        'answer':'''
        아 아이유는
        이 이상하게
        유 유난히 좋다.
        '''
    },
    {
        'question':'이순신으로 삼행시를 지어줘',
        'answer':'''
        이 이순신 장군은
        순 순순히 내어줄 것 같았던 조선의 바다를
        신 신의와 충의의 마음으로 지켜냈다.
        '''
    }
]


In [31]:
example_prompt=PromptTemplate(input_cariables=['question','answer'],template='question: {question}\nanswer: {answer}')

In [32]:
example_prompt.format(**examples[0]) #첫번째 템플릿

'question: 아이유로 삼행시를 지어줘\nanswer: \n        아 아이유는\n        이 이상하게\n        유 유난히 좋다.\n        '

In [33]:
prompt=FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix='question: {input}\nanswer:',
    input_variables=['input']
)

In [50]:
llm(prompt.format(input='유관순으로 삼행시를 지어줘'))

' \n        유 유관순은\n        관 관심 없는 시대에\n        순 순국향을 이끌었다.'